# Exploring eBay Car Sales

The aim of this project is to clean the data and analyze the included used car listings. You'll also become familiar with some of the unique benefits jupyter notebook provides for pandas.

The orriginally dataset we are working with is of Over 370000 used cars scraped with Scrapy from Ebay-Kleinanzeigen, you can find [here](https://data.world/data-society/used-cars-data).

The current dataset have some diffences:
- We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
- We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

## Exploring dataset
We'll start importing [numpy](https://numpy.org/) and [pandas](https://pandas.pydata.org/) to start work with the dataset.

In [1]:
import numpy as np
import pandas as pd

We'll use pandas [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html?highlight=read_csv#pandas-read-csv) function to read the dataset.

Note: We'll use `Latin-1` encoding for dataset.

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

We'll use [DataFrame.info()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html?highlight=info#pandas.DataFrame.info) and [DataFrame.head()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html?highlight=head#pandas.DataFrame.head) functions to start exploring the dataset  structure and the content.

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

- We can see that the columns `vehicleType`, `gearbox`, `model`, `fuelType` and `notRepairedDamage` contains null values.
- The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We can observe in the content of the first 5 columns.
- All colums (`dateCrawled`, `dateCreated`, `lastSeen`) that contain a date are actually datetimes.
- In the `price` column the amounts are formatted as US dollar, for example `$1,000.00`.
- In the `odometer` column the quantities contain the subfix of the unit of measure (`km`) and contain commas. 

## Snakecase for columns name
As we saw earlier, the column names are in `camelcase`, the first thing we will do is cleaning them to `snakecase` (Python convention). For that we'll use the [DataFrame.rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html?highlight=rename#pandas.DataFrame.rename) (with `columns` param to specify new columns names, and `inplace=True` arg to make change in current DataFrame).


For some columns we'll make an special rename of column:
- `yearOfRegistration` to `registration_year`
- `monthOfRegistration` to `registration_month`
- `notRepairedDamage` to `unrepaired_damage`
- `dateCreated` to `ad_created`
- `nrOfPictures` to `nr_pictures`

In [5]:
# The key are the current colum name
# The value are the new column name.
columns = {
    'dateCrawled': 'date_crawled',
    'offerType': 'offer_type',
    'vehicleType': 'vehicle_type',
    'powerPS': 'power_ps',
    'fuelType': 'fuel_type',
    'postalCode': 'postal_code',
    'lastSeen': 'last_seen',
    'yearOfRegistration': 'registration_year',
    'monthOfRegistration': 'registration_month',
    'notRepairedDamage': 'unrepaired_damage',
    'dateCreated': 'ad_created',
    'nrOfPictures': 'nr_pictures'
}

In [6]:
autos.rename(columns=columns, inplace=True)

## Initial data clening
Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:

- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
- Examples of numeric data stored as text which can be cleaned and converted

We'll use [DataFrame.describe()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html?highlight=describe#pandas.DataFrame.describe) (with include='all' to get both categorical and numeric columns) to look at descriptive statistics for all columns.

In [7]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-09 11:54:38,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


We can see the columns `seller`,`offer_type` have mostly one value, that are candidate to be dropped.

In [8]:
autos.value_counts('seller')

seller
privat        49999
gewerblich        1
dtype: int64

In [9]:
autos.value_counts('offer_type')

offer_type
Angebot    49999
Gesuch         1
dtype: int64

Any examples of numeric data stored as text that needs to be cleaned. We saw the `price` and `odometer` columns are numeric values stored as text. We'll remove any non-numeric character and convert column to numeric `dtype`.

We found the format for `price` column are `$1,000.00`, we need to remove the `$` and `,` characters.
We'll going to use:
- [Series.str](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) vectorized string function to replace the characters.
- [Series.astype()](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html) to cast to `int` dtype.


In [10]:
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(int)
autos.value_counts('price')

price
0           1421
500          781
1500         734
2500         643
1000         639
            ... 
8098           1
8099           1
8180           1
40990          1
99999999       1
Length: 2357, dtype: int64

We found the format for `odometer` column are `1,000km`, we need to remove the `km` and `,` characters. For this column we will also rename the column adding the suffix `_km` in order to not lose information of unit measure.


In [11]:
autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',', '').astype(int)
autos.value_counts('odometer')

odometer
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
dtype: int64

In [12]:
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

# Exploring the Odometer and Price Columns

We'll analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.

We'll use the next technique to expolring odometer and price columns:
- [Series.unique().shape](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html?highlight=unique#pandas.Series.unique()) to see how many unique values
- Series.describe() to view min/max/median/mean etc.
- Series.value_counts(), to see counts of unique values:

## Exploring the Odometer Column


In [13]:
odometer_km = autos['odometer_km']
odometer_km.unique().shape

(13,)

In [14]:
odometer_km.describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

We can use [Series.sort_index()](https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_index.html?highlight=sort_index#pandas.Series.sort_index) whit `ascending` param, to get the values_counts result ordered.

In [15]:
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

We can see that no value has any is very separate from another, and that it does not have a low frequency.

## Exploring the Price Column

In [16]:
price = autos['price']
price.unique().shape

(2357,)

In [17]:
price.describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

The `price` column has `2357` values, which makes it hard to analyze, we'll use [Series.head()](https://pandas.pydata.org/docs/reference/api/pandas.Series.head.html?highlight=head#pandas.Series.head) and [Series.tails()](https://pandas.pydata.org/docs/reference/api/pandas.Series.tail.html?highlight=tail#pandas.Series.tail) to see the top and last one's values.

In [18]:
price.value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

We can see that after the value 350,000  you start to see a very large separation of values.

In [19]:
price.value_counts().sort_index(ascending=False).tail(20)

35       1
30       7
29       1
25       5
20       4
18       1
17       3
15       2
14       1
13       2
12       3
11       2
10       7
9        1
8        1
5        2
3        1
2        3
1      156
0     1421
Name: price, dtype: int64

We tend to believe that cars priced at 0 are because the information was not available at the time. 

Therefore we'll only left the values between 1 and 350,000 

In [20]:
autos_clean_price = autos[price.between(1, 350000)]

## Exploring the date columns

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled` `(str)`: added by the crawler
- `last_seen` `(str)`: added by the crawler
- `ad_created` `(str)`: from the website
- `registration_month` `(int)`: from the website
- `registration_year` `(int)`: from the website

Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:

In [21]:
autos[['date_crawled','ad_created','last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


You'll notice that the first 10 characters represent the day (e.g. 2016-03-12).

To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

In [22]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

We can see that the range of the values is between the month of March and April of the year 2016.

In [23]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True).head(20)

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-01-03    0.00002
2016-01-07    0.00002
2016-01-10    0.00004
2016-01-13    0.00002
2016-01-14    0.00002
2016-01-16    0.00002
2016-01-22    0.00002
2016-01-27    0.00006
2016-01-29    0.00002
2016-02-01    0.00002
2016-02-02    0.00004
2016-02-05    0.00004
2016-02-07    0.00002
2016-02-08    0.00002
Name: ad_created, dtype: float64

In [24]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True).tail(20)

2016-03-19    0.03384
2016-03-20    0.03786
2016-03-21    0.03772
2016-03-22    0.03280
2016-03-23    0.03218
2016-03-24    0.02908
2016-03-25    0.03188
2016-03-26    0.03256
2016-03-27    0.03090
2016-03-28    0.03496
2016-03-29    0.03414
2016-03-30    0.03344
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03508
2016-04-03    0.03892
2016-04-04    0.03688
2016-04-05    0.01184
2016-04-06    0.00326
2016-04-07    0.00128
Name: ad_created, dtype: float64

We can see that there are a few records from the year 2015, most of the records are found between the month of January and April of the year 2016.

In [25]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
2016-03-15    0.01588
2016-03-16    0.01644
2016-03-17    0.02792
2016-03-18    0.00742
2016-03-19    0.01574
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-22    0.02158
2016-03-23    0.01858
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-26    0.01696
2016-03-27    0.01602
2016-03-28    0.02086
2016-03-29    0.02234
2016-03-30    0.02484
2016-03-31    0.02384
2016-04-01    0.02310
2016-04-02    0.02490
2016-04-03    0.02536
2016-04-04    0.02462
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092
Name: last_seen, dtype: float64

We can see that the range of the values is between the month of March and April of the year 2016.

Now we'll explore the `registration_year` column. The column has an int value representation of the year.

In [26]:
autos['registration_year'].head()

0    2004
1    1997
2    2009
3    2007
4    2003
Name: registration_year, dtype: int64

In [27]:
autos['registration_year'].value_counts().sort_index(ascending=False).head(20)

9999       4
9996       1
9000       2
8888       1
6200       1
5911       1
5000       4
4800       1
4500       1
4100       1
2800       1
2019       3
2018     492
2017    1453
2016    1316
2015     399
2014     666
2013     806
2012    1323
2011    1634
Name: registration_year, dtype: int64

In [28]:
autos['registration_year'].value_counts().sort_index(ascending=False).tail(30)

1963     9
1962     4
1961     6
1960    34
1959     7
1958     4
1957     2
1956     5
1955     2
1954     2
1953     1
1952     1
1951     2
1950     3
1948     1
1943     1
1941     2
1939     1
1938     1
1937     4
1934     2
1931     1
1929     1
1927     1
1910     9
1800     2
1500     1
1111     1
1001     1
1000     1
Name: registration_year, dtype: int64

We can observe outliers
- Values greater than 2016, it is unlikely that there will be more recent models this year since all date crawled are less than or equal to 2016
- Probably car registrations older than 1990 models, we can ignore values lower than 1958 because they have a  low frequency

In [29]:
autos = autos[autos['registration_year'].between(1958, 2016)]
autos['registration_year'].value_counts(normalize=True).sort_index(ascending=False)

2016    0.027425
2015    0.008315
2014    0.013879
2013    0.016797
2012    0.027571
2011    0.034052
2010    0.033281
2009    0.043721
2008    0.046493
2007    0.048014
2006    0.056433
2005    0.062831
2004    0.057037
2003    0.056829
2002    0.052786
2001    0.056329
2000    0.069895
1999    0.062518
1998    0.051119
1997    0.042262
1996    0.030092
1995    0.027362
1994    0.013754
1993    0.009274
1992    0.008148
1991    0.007419
1990    0.008232
1989    0.003772
1988    0.002959
1987    0.001563
1986    0.001584
1985    0.002188
1984    0.001104
1983    0.001104
1982    0.000896
1981    0.000646
1980    0.002021
1979    0.000729
1978    0.000979
1977    0.000458
1976    0.000563
1975    0.000396
1974    0.000500
1973    0.000542
1972    0.000729
1971    0.000563
1970    0.000938
1969    0.000396
1968    0.000542
1967    0.000563
1966    0.000458
1965    0.000354
1964    0.000250
1963    0.000188
1962    0.000083
1961    0.000125
1960    0.000709
1959    0.000146
1958    0.0000

In [30]:
autos['registration_year'].describe()

count    47986.000000
mean      2002.859897
std          7.043243
min       1958.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

In [31]:
autos['registration_year'].value_counts(normalize=True).sort_index(ascending=False).head(20).sum()

0.8675863793606469

We can see 86% of car linting are models between 1997 and 2106

In [32]:
autos[autos['registration_year'].between(1997, 2016)]['registration_year'].describe()

count    41632.000000
mean      2004.653920
std          5.022961
min       1997.000000
25%       2000.000000
50%       2004.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

## Exploring Price by Brand

In [33]:
brands = autos['brand']
brands.value_counts(normalize=True).head(10).sum()

0.8030884007835618

We'll choose the first ten brands ordered because they represent 80% of all brands in dataset.


Series.value_counts() produces a series with index labels, so we'll use Series.index attribute to access the brand labels.

In [34]:
selected_brands = brands.value_counts().head(10).index
selected_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat'],
      dtype='object')

We'll use aggregation analysis techniques:
- Identify the unique values we want to aggregate by
- Create an empty dictionary to store our aggregate data
- Loop over the unique values, and for each: - Subset the dataframe by the unique values - Calculate the mean of whichever column we're interested in - Assign the val/mean to the dict as k/v.

In [35]:
brand_mean_prices = {}
for brand in selected_brands:
    brand_mean_prices[brand] = autos[brands == brand]['price'].mean()
    
brand_mean_prices

{'volkswagen': 6512.984195543339,
 'bmw': 8334.045996592846,
 'opel': 5253.836062415719,
 'mercedes_benz': 30296.012461740273,
 'audi': 9093.65003615329,
 'ford': 7238.837813620072,
 'renault': 2396.2067751869777,
 'peugeot': 3039.4682651622,
 'fiat': 2711.8011272141707,
 'seat': 4301.417431192661}

We can see from the selected brands, on average the most expensive are `mercedes_benz`, `audi`, `bmw` and the cheapest are `peugeot`, `fiat`, `renault`.

We'll use [pd.Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) constructor to make new series from `brand_mean_prices`.

In [36]:
brand_mean_prices_series = pd.Series(brand_mean_prices)
brand_mean_prices_series

volkswagen        6512.984196
bmw               8334.045997
opel              5253.836062
mercedes_benz    30296.012462
audi              9093.650036
ford              7238.837814
renault           2396.206775
peugeot           3039.468265
fiat              2711.801127
seat              4301.417431
dtype: float64

Then We'll use [pd.Dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) constructor with columns param from series `brand_mean_prices_series`.

In [37]:
df = pd.DataFrame(brand_mean_prices_series, columns=['mean_price'])
df

,mean_price
volkswagen,6512.984196
bmw,8334.045997
opel,5253.836062
mercedes_benz,30296.012462
audi,9093.650036
ford,7238.837814
renault,2396.206775
peugeot,3039.468265
fiat,2711.801127
seat,4301.417431


## Exploring Mileage by Brand

In [38]:
brand_mean_mileage = {}
for brand in selected_brands:
    brand_mean_mileage[brand] = autos[brands == brand]['odometer_km'].mean()
    
brand_mean_mileage

{'volkswagen': 128733.18935898694,
 'bmw': 132458.8302101079,
 'opel': 129258.33172799075,
 'mercedes_benz': 130956.4932225623,
 'audi': 129287.78018799711,
 'ford': 124168.16009557946,
 'renault': 128238.01143862736,
 'peugeot': 127136.81241184767,
 'fiat': 116553.94524959743,
 'seat': 121697.24770642201}

In [39]:
brand_mean_mileage_series = pd.Series(brand_mean_mileage)
brand_mean_mileage_series

volkswagen       128733.189359
bmw              132458.830210
opel             129258.331728
mercedes_benz    130956.493223
audi             129287.780188
ford             124168.160096
renault          128238.011439
peugeot          127136.812412
fiat             116553.945250
seat             121697.247706
dtype: float64

We'll assign the `brand_mean_mileage_series` series as a new column in this dataframe.


In [40]:
df['mean_mileage_km'] = brand_mean_mileage_series
df

,mean_price,mean_mileage_km
volkswagen,6512.984196,128733.189359
bmw,8334.045997,132458.830210
opel,5253.836062,129258.331728
mercedes_benz,30296.012462,130956.493223
audi,9093.650036,129287.780188
ford,7238.837814,124168.160096
renault,2396.206775,128238.011439
peugeot,3039.468265,127136.812412
fiat,2711.801127,116553.945250
seat,4301.417431,121697.247706


In [41]:
df.sort_values(ascending=False, by='mean_price')

,mean_price,mean_mileage_km
mercedes_benz,30296.012462,130956.493223
audi,9093.650036,129287.780188
bmw,8334.045997,132458.830210
ford,7238.837814,124168.160096
volkswagen,6512.984196,128733.189359
opel,5253.836062,129258.331728
seat,4301.417431,121697.247706
peugeot,3039.468265,127136.812412
fiat,2711.801127,116553.945250
renault,2396.206775,128238.011439


We can see a all models have approximately the same mileage

## Cleaning next steps

We'll start by convert the dates to be uniform numeric data, so "2016-03-21" becomes the integer 20160321. The columns we'll covert are:
 - `date_crawled`
 - `ad_created`
 - `last_seen`

In [42]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [43]:
autos['date_crawled']= autos['date_crawled'].str[:10].str.replace('-','').astype(int)
autos['ad_created'] = autos['ad_created'].str[:10].str.replace('-','').astype(int)
autos['last_seen'] = autos['last_seen'].str[:10].str.replace('-','').astype(int)

autos[['date_crawled', 'ad_created', 'last_seen']].head()

,date_crawled,ad_created,last_seen
0,20160326,20160326,20160406
1,20160404,20160404,20160406
2,20160326,20160326,20160406
3,20160312,20160312,20160315
4,20160401,20160401,20160401


Next we'll see if there are particular keywords in the name column that you can extract as new columns

In [44]:
autos['name'].value_counts()

Ford_Fiesta                                       76
Volkswagen_Golf_1.4                               75
BMW_316i                                          75
Volkswagen_Polo                                   72
BMW_318i                                          72
                                                  ..
Skoda_Roomster_mit_AHK_und_allem_Komfort           1
Mercedes_Benz_E_220_CDI_Automatik_Classic_W211     1
Golf_4_1.6_sr_Klimatronik_ele_Fenster              1
Mercedes_Benz__C_200                               1
Renault_Laguna_mit_Tuev                            1
Name: name, Length: 36982, dtype: int64

We can see that the name does not maintain a specific pattern with which it can work, the name values include information about the model and engine of the car, the dataset already has a column with the model car.

## Exploring most common Model.

We'll find the most common brand/model combinations:
- First, we'll use pd.Dataframe.groupby method to make the aggregations for brand and model columns.
- Then we'll use pd.DataFrameGroupBy.size method to make the count of aggregations.
- Last we'll use pd.DataFrame.sort_values method.

In [45]:
autos.groupby(["brand", 'model']).size().sort_values(ascending=False)

brand       model     
volkswagen  golf          3815
bmw         3er           2688
volkswagen  polo          1677
opel        corsa         1644
            astra         1388
                          ... 
audi        200              1
rover       discovery        1
            rangerover       1
bmw         i3               1
ford        b_max            1
Length: 290, dtype: int64

In [46]:
autos.groupby(["brand", 'model'])

The most common car is the `Volkswagen` `Golf`.

## Exploring Mileage vs Price

We'll split the `odometer_km` into groups, and use aggregation to see if average prices follows any patterns based on the mileage.

In [47]:
autos['odometer_km'].describe()

count     47986.000000
mean     125611.011545
std       40035.616409
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [48]:
mileage = autos['odometer_km'].unique()
mileage.sort()
mileage

array([  5000,  10000,  20000,  30000,  40000,  50000,  60000,  70000,
        80000,  90000, 100000, 125000, 150000])

We; ll define four groups of mileage, stating for 50000 was the min value of `odometer_km`, to define each group we sum the standard deviation value and selecting the value of the close of the unique values of  `odometer_km`

In [49]:
mileage_groups = [(5000, 40000), (50000, 70000), (80000, 100000), (125000, 150000)]

In [50]:
mileage_mean_price = {}
for group in mileage_groups:
    mileage_mean_price[f'{group[0]} - {group[1]}'] = autos[autos['odometer_km'].between(group[0],group[1])]['price'].mean()

mileage_mean_price

{'5000 - 40000': 23484.200576368876,
 '50000 - 70000': 15857.369682444578,
 '80000 - 100000': 10412.213749277875,
 '125000 - 150000': 7536.564624148951}

We can see that cars with lower mileage on average are more expensive.  

## Exploring price of car damage vs non-damaged.

In [55]:
not_rep_dam = {'nein': 'no',
               'ja': 'yes',
               np.nan : 'unknown'}
autos['unrepaired_damage'] = autos['unrepaired_damage'].map(not_rep_dam)


In [52]:
autos['unrepaired_damage'].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [53]:
autos[autos['unrepaired_damage']=='yes']['price'].mean()

2317.5163248221015

In [54]:
autos[autos['unrepaired_damage']=='no']['price'].mean()

8223.002161214954

We can see that on average the cars non-damage are more expensive.